### Setup

In [1]:
import pandas as pd
import numpy as np
import os
from time import time

In [2]:
#import sys
#sys.path.append(os.path.abspath('..'))#sys.path[0] + '/..') 
from vb_estimators import  LinRegSupreme,LinSVR,RBFSVR,ENet,L1Lars,GBR,HGBR,FlexiblePipe
from vb_helper import VBHelper
from vb_cross_validator import regressor_q_stratified_cv

no daal4py


In [3]:

from warnings import filterwarnings
filterwarnings('ignore')


### setup the experiment/project
#### note the 'run_stacked' kwarg that can be set to create the stacked_regressor

In [4]:
gridpoints=5
kwargs=dict(
    shuffle=True,
    run_stacked=True,
    test_share=0,#keep at 0 for small datasets
    cv_folds=5,
    cv_reps=10,
    #cv_groupcount=5,
    cv_strategy=('quantile',5), # for stratified cv
    random_state=2 # random_state for reproducibility
)
vbhelper=VBHelper(**kwargs)

In [5]:
scorer_list=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'] #cross_validate wants strings
vbhelper.scorer_list=scorer_list

##### User Import Dataset Step

In [6]:
data_path=os.path.join('sample_data','ex1.csv')
df=pd.read_csv(data_path)
all_vars=list(df.columns)
print(all_vars)

['STA_ID', 'LONG', 'LAT', 'OrigHabCode', 'Date', 'THG_Fish', 'YEAR', 'SEASON', 'SUBAREA', 'HABCODE', 'Floc_Depth_ft', 'AFDW_Floc', 'MEHG_Floc', 'THG_floc', 'Tot_Phos_floc', 'Bulk_Dens_Floc', 'Soil_Thickness_FT', 'AFDW_Soil', 'Bulk_Dens_Soil', 'PH_soil', 'SO4_soil', 'MEHG_soil', 'THG_soil', 'Tot_Carbon_Soil_%', 'Tot_Nitrogen_Soil_%', 'Tot_Phos_soil', 'Wat_Depth_ft', 'COND_SW', 'DO_SW', 'TEMP_SW', 'PH_SW', 'TURB_SW', 'REDOX_SW', 'Alk_Phos_SW', 'CHLA_SW', 'CL_SW', 'MEHG_SW', 'NH4_SW', 'NO2_SW', 'NO3_SW', 'SO4_SW', 'Sol_Reac_Phos_SW', 'THG_SW', 'TOC_SW', 'Tot_Nitrogen_SW', 'Tot_Phos_SW', 'REDOX_PW', 'H2S_PW', 'Sol_Reac_Phos_PW', 'MEHG_Peri_AVG', 'THG_epi_peri']


#### user has option to specify "regulatory standard"

In [7]:
#The user sets the variables to use for x and y.
y_name='THG_Fish'
loc_vars=['LAT','LONG']
drop_vars=['Date','OrigHabCode','STA_ID']
drop_vars.extend(loc_vars)
drop_vars.append(y_name)
x_vars=[var for var in all_vars if var not in drop_vars]
X_df=df.loc[:,x_vars]
y_df=df.loc[:,y_name].to_frame()

In [8]:
vbhelper.setData(X_df,y_df)

# of duplicate rows of data: 0
# of duplicate rows of X: 0


### setup the analytical pipelines
#### note the inner_cv_dict and prep_dict that are used to consolidate vb_estimator kwargs and to facilitate the divison between prep and post steps.

In [9]:
inner_cv_dict={'cv_reps':1,'cv_folds':5,'cv_strategy':('quantile',5)}
inner_cv=vbhelper.getCV(cv_dict=inner_cv_dict)

prep_dict={'impute_strategy':'impute_knn5','cat_idx':vbhelper.cat_idx}

pipe_kwargs=dict(do_prep=not vbhelper.run_stacked,prep_dict=prep_dict,inner_cv=inner_cv,gridpoints=gridpoints,cat_idx=vbhelper.cat_idx,float_idx=vbhelper.float_idx,bestT=False)
pipe_dict={
    'gradient-boosting-reg':{'pipe':GBR,'pipe_kwargs':dict(
        prep_dict=prep_dict,do_prep=not vbhelper.run_stacked)},
    #'lin-reg-supreme':{'pipe':LinRegSupreme,'pipe_kwargs':pipe_kwargs}, 
    #'powXB-least-sq':{'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'flex_kwargs':{'form':'powXB'}}}),
    #'expXB-least-sq':{'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'flex_kwargs':{'form':'expXB'}}}), #expXB is default
    #'nonlinear-search-least-sq': {'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'functional_form_search':True}},
    #'robust-powXB-least-sq':{'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'flex_kwargs':{'form':'powXB','robust':True}}},
    #'robust-expXB-least-sq':{'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'flex_kwargs':{'form':'expXB','robust':True}}}, #expXB is default
    #'robust-nonlinear-search-least-sq': {'pipe':FlexiblePipe,'pipe_kwargs':{**pipe_kwargs,'functional_form_search':True,'flex_kwargs':{'robust':True}}},
    #'histogram-gradient-boosting-reg':{'pipe':HGBR,'pipe_kwargs':{'prep_dict':{'cat_idx':vbhelper.cat_idx}}},
    
    
    #'elastic-net':{'pipe':ENet,'pipe_kwargs':pipe_kwargs}, 
    #'linear-svr-cv':{'pipe':LinSVR,'pipe_kwargs':pipe_kwargs}
    'rbf-svr-cv':{'pipe':RBFSVR,'pipe_kwargs':pipe_kwargs}, 
    'lassolars':{'pipe':L1Lars,'pipe_kwargs':pipe_kwargs},
    }


#estimator_dict={'multi_pipe':{'pipe':MultiPipe(pipelist=[(k,v) for k,v in estimator_dict.items()],cat_idx=vbhelper.cat_idx)}
vbhelper.setPipeDict(pipe_dict) #formerly setEstimatorDict
vbhelper.setModelDict()
#vbhelper.model_dict={key:val() for key,val in vbhelper.estimator_dict.items()} # they will be models once .fit is called

### create a smaller test run to check runtime, debug, etc.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_df,y_df,test_size=0.5,random_state=5) #just for debugging

In [11]:
runtest=True
if runtest:
    for name,est in vbhelper.model_dict.items():
        start=time()
        print(name)
        est.fit(X_train,y_train)
        print('train R2:',est.score(X_train,y_train))
        if not X_test is None:print('test R2:',est.score(X_test,y_test))
        end=time()
        print(f'runtime:{(end-start)/60} min.\n')


multi_pipe
train R2: 0.6393890104294966
test R2: 0.46776458502285223
runtime:0.590310295422872 min.



In [12]:
if vbhelper.run_stacked and runtest:
    mp=vbhelper.model_dict['multi_pipe']
    fitted_ipipe_dict=mp.build_individual_fitted_pipelines()
    print(fitted_ipipe_dict['lassolars'].score(X_test,y_test))
    print(fitted_ipipe_dict['gradient-boosting-reg'].score(X_test,y_test))
    print(fitted_ipipe_dict['rbf-svr-cv'].score(X_test,y_test))
    
    

0.3978539049810762
0.3323798670465754
0.42592350110871346


### end small test-run debugging

---

### Fit the final models for all estimators

In [13]:
vbhelper.runCrossValidate(try_load=True) #try_load speeds things up by reloading results if they've been run before with same setup and data

jhash:  cec6a473dd42cba3c8c208c0c24d2316
existing cv_results loaded


In [14]:
#vbhelper.fitFinalModelDict()

In [15]:
#plot cv_yhat

#### graphs and table to summarize results

In [16]:
vbhelper.buildCVScoreDict()

In [17]:
vbhelper.cv_score_dict

{'gradient-boosting-reg': {'neg_mean_squared_error': array([-8027.27661867, -7887.01583286, -7982.66163052, -7993.93961746,
         -8189.47426764, -7991.32725446, -7528.36135885, -7970.75893139,
         -8241.4365886 , -7766.71661619]),
  'neg_mean_absolute_error': array([-63.90688711, -64.4972363 , -63.91561586, -64.30253942,
         -64.73073203, -63.40476397, -62.64060512, -63.9805329 ,
         -65.13061583, -64.00156006]),
  'r2': array([0.4082103 , 0.41855065, 0.41149942, 0.41066798, 0.39625271,
         0.41086057, 0.44499151, 0.41237692, 0.39242192, 0.4274194 ])},
 'rbf-svr-cv': {'neg_mean_squared_error': array([-7776.76246669, -7905.14131381, -7909.10392396, -7763.24616289,
         -7928.58253935, -7777.16359077, -7835.92082854, -7672.20807157,
         -8068.08957136, -7929.1301927 ]),
  'neg_mean_absolute_error': array([-60.23137578, -61.46848975, -61.01620971, -60.26292807,
         -60.96825113, -59.90130232, -61.59578668, -59.95660507,
         -61.73273606, -61.6634

In [18]:
vbhelper.viewCVScoreDict()

scores for scorer: neg_mean_squared_error:
    multi_pipe:-7144.933854820153
scores for scorer: neg_mean_absolute_error:
    multi_pipe:-60.79993325706245
scores for scorer: r2:
    multi_pipe:0.4732586819154962


In [19]:
vbhelper.predictCVYhat()

In [20]:
#vbhelper.jsonifyProjectCVResults()

In [21]:
vbhelper.cv_score_dict

{'gradient-boosting-reg': {'neg_mean_squared_error': array([-8027.27661867, -7887.01583286, -7982.66163052, -7993.93961746,
         -8189.47426764, -7991.32725446, -7528.36135885, -7970.75893139,
         -8241.4365886 , -7766.71661619]),
  'neg_mean_absolute_error': array([-63.90688711, -64.4972363 , -63.91561586, -64.30253942,
         -64.73073203, -63.40476397, -62.64060512, -63.9805329 ,
         -65.13061583, -64.00156006]),
  'r2': array([0.4082103 , 0.41855065, 0.41149942, 0.41066798, 0.39625271,
         0.41086057, 0.44499151, 0.41237692, 0.39242192, 0.4274194 ])},
 'rbf-svr-cv': {'neg_mean_squared_error': array([-7776.76246669, -7905.14131381, -7909.10392396, -7763.24616289,
         -7928.58253935, -7777.16359077, -7835.92082854, -7672.20807157,
         -8068.08957136, -7929.1301927 ]),
  'neg_mean_absolute_error': array([-60.23137578, -61.46848975, -61.01620971, -60.26292807,
         -60.96825113, -59.90130232, -61.59578668, -59.95660507,
         -61.73273606, -61.6634

In [22]:
assert False, 'plots moved elsewhere'

AssertionError: plots moved elsewhere

In [ ]:
#vbhelper.pickleSelf() #for development in other notebooks

In [ ]:
vbhelper.plotCVScores(sort=1)

In [ ]:
vbhelper.plotCVYhatVsY(regulatory_standard=False,decision_criteria=False)

#### Add table of results for the pipelines, coefficients, stats, etc. 

In [ ]:
vbhelper.plotCVYhat(single_plot=True) 
# make Y line faint and make dots at actual values
# add the mean of each series and make more visible
# sort by original row order or by increasing value of Y

In [ ]:
vbhelper.plotCVYhat(single_plot=False)

### Sensitivity analysis of rows
Outlier plots
DfFits, leverage

### Sensitivity analysis of columns/features
#### partial dependence plots (PDP in GBM)

## Next, user selects pipeline, final model is fit and ready for prediction

In [ ]:
###